# Load Packages

In [1]:
%matplotlib inline 
from ifis_tools import database_tools as db 
from ifis_tools import asynch_manager as am 
from ifis_tools import auxiliar as aux
from wmf import wmf
import pandas as pd 
import numpy as np 
import os 
import pylab as pl
from string import Template

No se logra importar deap tools, por lo tanto se deshabilita SimuBasin.Calib_NSGAII


# Define watershed from database

In [3]:
#Get the watershed
cu = am.hlmModel(linkid=272678, ExtraParams=['h_order'])

#Temporal rvr to get the topology of the watershed
cu.write_rvr('data/for_hlm/south_skunk.rvr')
topo = cu.topo.copy()
topo.set_index('link_id', inplace = True)
idx = cu.Table.index.intersection(topo.index)
cu.Table['dest'] = topo['id'].loc[idx]

#Read the coordinates 
coor,d = wmf.read_map_points('data/maps/hills_centroids_proj.shp', ['link_id'])
Coor = pd.DataFrame(coor.T, index=d['link_id'])
Coor.rename(columns={0:'X',1:'Y'}, inplace=True)

In [11]:
#export
def Get_sub_watersheds(watershed, order_max, order_min = 4):
    '''Obtains the sub-watersheds a different orders starting from the order_max and 
    ending on the order_min, there is no return, it only updates the watershed.Table'''
    orders = np.arange(order_max, order_min, -1).tolist()
    for Prun in orders:
        #Finds the connections points
        Ho = Prun
        watershed.Table['prun_'+str(Ho)] = 0
        idx = watershed.Table.loc[watershed.Table['h_order']>=Ho].index
        for i in idx:
            size = watershed.Table.loc[(watershed.Table['dest'] == i) & (watershed.Table['h_order'] >= Ho-1)]
            if size.shape[0] >= 2:
                watershed.Table.loc[size.index, 'prun_'+str(Ho)] = 1

        #Finds all the links that belong to a pruning level
        idx = watershed.Table.loc[watershed.Table['prun_'+str(Ho)] == 1].sort_values(by = ['Acum'], ascending = False).index
        cont = 2
        for i in idx:
            #Finds the watershed upstream
            t = am.hlmModel(linkid=i)
            idx_t = watershed.Table.index.intersection(t.Table.index)
            #Assign that pruning level to the sub-watershed
            watershed.Table.loc[idx_t, 'prun_'+str(Ho)] = cont
            #Go to next pruning level
            cont += 1
        print('Prun %d done' % Prun)

In [14]:
Get_sub_watersheds(cu, 7, 5)

Prun 7 done
Prun 6 done
